In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from termcolor import colored

In [2]:
from gensim import models
model = models.Word2Vec.load('../model/fbcomment.w2v')

Using TensorFlow backend.


In [3]:
df = pd.read_csv("fb_comment_token_annotated.tsv",encoding='utf-8',sep='\t')
df['token_arr'] = df['token'].apply(lambda x:eval(x))

In [294]:
comments = df['token_arr'].apply(lambda x: " ".join(x))
labels = df['judgeScore']
for i,comment in enumerate(comments):
    label = labels[i]
    row = ''
    row+="__label%d__ "%(int(label))
    row+=comment.replace("\n"," ")
    row+="\n"
    
    with open("./train2fastText.txt", "a") as myfile:
        myfile.write(row)

In [4]:
scored = df[df['judgeScore']!=-32]
scored['wordNum']= scored['token_arr'].apply(lambda x: len(x))


/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
scored[scored['wordNum']<75]['message'].shape

(6244,)

In [6]:
#Let's use 15 as max_seq_len
max_seq_len = 75
k = 800
lessThanMaxSeqLen = scored['wordNum']<=max_seq_len
scoreddf = scored[lessThanMaxSeqLen]
X = scoreddf['token_arr'].values
y = (scored['judgeScore'][lessThanMaxSeqLen]+2)/4

In [7]:
X_train,X_test , y_train,y_test = train_test_split(X,y.values,test_size=0.2,random_state=42)
X_test,X_valid , y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,random_state=42)
print("Train set Row: %d"%(X_train.shape[0]))
print("Test set Row: %d"%(X_test.shape[0]))
print("Validate set Row: %d"%(X_valid.shape[0]))

Train set Row: 5004
Test set Row: 626
Validate set Row: 626


In [8]:
y.value_counts()

0.250    2758
0.750    1466
0.500    1181
0.000     335
1.000     259
0.125     171
0.875      86
Name: judgeScore, dtype: int64

In [9]:
mapping = [-1,0,1]
def generate_batch(i,batch_size=32,verbose=False,X=X_train,y=y_train):
    comments = X[i:i+batch_size] 
    avaiRow = comments.shape[0]
    batch_X = np.zeros([batch_size,max_seq_len,k+2])
    batch_Y = y[i:i+batch_size]
    for i,comment in enumerate(comments):
        for j in range(max_seq_len):
            if(max_seq_len-j>len(comment)): # Should not be word yet
                if(verbose):print("PAD",end=' ')
                batch_X[i][j][-1] = 1
            else: 
                if(verbose):print(comment[j-(max_seq_len-len(comment))],end=' ')
                try:
                    batch_X[i][j] = np.concatenate((model[comment[j-(max_seq_len-len(comment))]],np.zeros([2])))
                except Exception:
                    if(verbose):print("UNK",end=' ')
                    batch_X[i][j] = np.concatenate((np.zeros([k+1]),[1]))
        if(verbose):print(batch_Y[i])
    if(verbose):print(i)
    batch_Y = np.array([np.array([i]) for i in batch_Y])
    return (batch_X[:avaiRow],batch_Y[:avaiRow])

import random
(batch_X,batch_Y) = generate_batch(random.randint(0,X_train.shape[0]),verbose=True)

PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD อยาก ผ่อน แต่ ไม่ มี บัตร ทำไง คร้า 0.75
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD ขี้ กาก มาก ๆ ๆ ๆ เน็ต ดี แทค ที่ พิษณุโลก ห่วย สุด ๆ ค่าย อื่น ดี กว่า อีก แรง ไม่ มี ตก
กาก UNK ๆ ๆ ๆ ๆ ๆ ๆๆๆๆๆๆๆๆๆ UNK ๆๆๆ ๆ ๆ ๆ ๆๆๆ ๆ
 เสียดาย เงิน ที่ เติม ไป 0.25
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD Huawei   p10   กับ   p10 +   ลด เท่า ไร ครับ 0.75
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 

In [10]:
#Prepare test and validate set
(test_X,test_Y) = generate_batch(0,10000,X=X_test,y=y_test)
(valid_X,valid_Y) = generate_batch(0,10000,X=X_valid,y=y_valid)
print(test_X.shape)
print(test_Y.shape)

(626, 75, 802)
(626, 1)


## Create a model

In [11]:
import tensorflow as tf
#Define two variables which will hols the input data and the target data.
data = tf.placeholder(tf.float32, [None, max_seq_len,k+2])
target = tf.placeholder(tf.float32, [None, 1])

In [12]:
#Create RNN cell
from tensorflow.python.ops import rnn, rnn_cell
num_hidden = 200
num_layers = 3
cells = []
for i in range(num_layers):
    cell = rnn_cell.BasicLSTMCell(num_hidden,state_is_tuple=True)
    cell2 = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.5)
    cells.append(cell2)
cell = rnn_cell.MultiRNNCell(cells , state_is_tuple=True)

In [13]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
valT = tf.transpose(val,[1,0,2])
last = tf.gather(valT,int(valT.get_shape()[0])-1)

In [14]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.sigmoid(tf.matmul(last,weight)+bias)

In [15]:
allpredictions = tf.transpose(tf.map_fn(lambda x: tf.nn.sigmoid(tf.matmul(x,weight)+bias), valT, dtype=tf.float32),[1,0,2])

In [16]:
loss = tf.reduce_mean(tf.squared_difference(prediction, target))

In [17]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(loss)

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [39]:
init_op = tf.global_variables_initializer()

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.Session(config=config)

sess.run(init_op)

In [19]:
def toSent(n):
    if(n<0.45):return 0
    elif(n<0.65):return 0.5
    else:return 1

In [20]:
def cal_acc(pred,real):
    assert(pred.shape[0]==real.shape[0])
    pred_label = pred.copy()
    pred_label[(pred_label<0.45)] = 0
    pred_label[(pred_label>0.45)&(pred_label<0.65)] = 0.5
    pred_label[(pred_label>.65)&(pred_label<=1)] = 1

    real_label = real.copy()
    real_label[(real_label<0.45)] = 0
    real_label[(real_label>0.45)&(real_label<0.65)] = 0.5
    real_label[(real_label>0.65)&(real_label<=1)] = 1
    return ((pred_label==real_label).sum())/pred_label.shape[0]


In [21]:
batch_size = 32
no_of_batches = int(len(X_train)/batch_size)
epoch = 30
def printSegmentColor(word,sent):
    if(sent<0.2): return print(colored(word, 'red',attrs=['bold']),end='')
    elif(sent<0.45): return print(colored(word, 'red'),end='')
    elif(sent<0.65): return print(colored(word, 'grey'),end='')
    elif(sent<0.80): return print(colored(word, 'blue'),end='')
    elif(sent<=1): return print(colored(word, 'blue',attrs=['bold']),end='')

    
for i in range(epoch):
    ptr = 0
    avgAcc = 0
    for j in range(no_of_batches):
        #inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        (batch_X,batch_Y) = generate_batch(ptr)
        ptr+=batch_size
        feed_dict = {data: batch_X, target: batch_Y}
        sess.run(minimize,feed_dict)
        predict = sess.run(prediction,feed_dict)
        avgAcc+= cal_acc(predict,batch_Y)/no_of_batches
        #accuracy = sess.run(acc,{data:test_X,target:test_Y})
        #avgAcc += accuracy/no_of_batches
        print(".",end='')
        
    print("\nEpoch - ",str(i))
    print("Avg Accuracy (training set): %.2f"%avgAcc)
    #print("Avg acc: {}".format(round(avgAcc,2)))
    selInt = random.randint(0,test_X.shape[0]-11)
    test_pred = sess.run(prediction,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    all_test_pred = sess.run(allpredictions,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    err = sess.run(loss,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    #print("Loss: %f"%err)
    
    validation_pred = sess.run(prediction,{data: test_X, target: test_Y})
    acc = cal_acc(validation_pred,test_Y)
    print("Accuracy (test set): %.2f"%acc)
    print("====================================")
    print("Show test result")
    correct = 0
    for i in range(len(test_pred)):
        pred = test_pred[i]
        current_pred_seq = all_test_pred[i][-len(X_test[selInt+i]):]
        #Prin word with color by prediction
        for j,word in enumerate(X_test[selInt+i]):
            word_pred = current_pred_seq[j]
            printSegmentColor(word,word_pred)
        print(", %.2f Ans: %.2f"%(pred[0],y_test[selInt+i]),end=' ')
        if(toSent(pred[0])==toSent(y_test[selInt+i])): 
            print("✓")
            correct+=1
        else: 
            print("X")
    print("Acc: %.2f"%(correct/10))
validation_pred = sess.run(prediction,{data: valid_X, target: valid_Y})
incorrect = sess.run(loss,{data: valid_X, target: valid_Y})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))


............................................................................................................................................................
Epoch -  0
Avg Accuracy (training set): 0.62
Accuracy (test set): 0.59
Show test result
โปรสุดยอดอะเปลี่ยนใช้ทรูดีกว่ามั้ง, 0.68 Ans: 0.25 X
ทั้ง 2 แพ็คลดสปีดมั้ยครับ, 0.71 Ans: 0.50 X
ไม่ใช่คนไทยซื้อเครื่องศูนย์แบบรายเดีอนasus 6ได้ไหมครัย, 0.84 Ans: 0.75 ✓
ๆม่รับผิดชอบ อย่าใช้ๆเลย
เน็ตครบปริมาณแต่มีแถมอีกแต่เสือกใช้งานแทบไม่ได้, 0.29 Ans: 0.25 ✓
ผมสมัครไปแล้วเมื่ออาทิตที่แล้วจะมีสิทธิได้รับไมครับ, 0.70 Ans: 0.50 X
โปร 4  m งามใส้เลยเสียดายเงินมาก, 0.50 Ans: 0.25 X
สัญญาณกากมาก..ขนาดใน กทมในห้างใหญ่ๆแม่งยังกาก...วันจันนี้ย้ายแน่นอน, 0.02 Ans: 0.00 ✓
iphone 5 เท้าไรคับ ไม่ต้องมีโปรโมชั่น เครื่องเปล่าเท่าไรคับ, 0.87 Ans: 0.75 ✓
เจ๊จัดหนัก#dtacreward aunchasa kullavijit, 0.99 Ans: 0.75 ✓
สั่งชื้อได้แบบไหนค่ะ2500นะค่ะใช้AISยุค่ะ, 0.77 Ans: 0.75 ✓
Acc: 0.60
................................................................................

............................................................................................................................................................
Epoch -  4
Avg Accuracy (training set): 0.75
Accuracy (test set): 0.68
Show test result
นี่คืออะไร ไม่ได้เปิดเน็ต เปิดเครือข่ายมือถือ แต่โดนหักเงิน เสียเงินมาหลายรอบแล้ว ขอคำอธิบายด่วน ก่อนที่อีกหลายๆเบอร์จะเลิกใช้, 0.17 Ans: 0.50 X
Sirawich Voungchuyเปลี่ยนมาใช้โปรนี้กัน, 0.66 Ans: 0.50 X
ผมจะขอกรุณาเช็คขอมูลการโทรของเบอร์ตัวเองจะได้หรือปล่าวครับ ใน in box ไม่มีใครพอจะตอบให้ผมได้เลย เดี๋ยวผมย้ายค่ายหนีนะครับ, 0.25 Ans: 0.25 ✓
รักเลย #DTAC4G ❤️❤️❤️, 0.99 Ans: 1.00 ✓
ใช้ดีแทคมานาน และไม่คิดจะย้ายค่าย แต่พอมาเจอเน็ตห่วยแตกแล้ว ย้ายค่ายดีกว่า เสียดายเงินที่สมัคร เน็ตเล่นเห้ยอะไรไม่ได้เลย, 0.24 Ans: 0.25 ✓
แจกถึงสี่โมงเย็น คือ ยังไม่เลิกงานเลย กะแจกคนที่เค้าว่างๆ ไม่ก็ทำงานในห้างใช่มะ จัดโปรแแบกลัวคนไปเอาของแจกเยอะอย่าทำเลย, 0.23 Ans: 0.25 ✓
แบตมันจะเสื่อมไวไหมครับ, 0.56 Ans: 0.50 ✓
ยังมีเครื่องอยู่มั้ยครับ, 0.66 Ans: 0.75 ✓
นี่ขนาดวาย

............................................................................................................................................................
Epoch -  7
Avg Accuracy (training set): 0.82
Accuracy (test set): 0.66
Show test result
ประเทศพม่ารหัรเลขอะไร, 0.17 Ans: 0.50 X
อย่าดีใจไป มันคือราคาผ่อนต่อเดือนเจ้าค่ะ, 0.40 Ans: 0.50 X
อยาก ดู มาก Vijai Pongpeerayanant #dtacreward, 0.58 Ans: 0.75 X
เน็ตกากมาก, 0.20 Ans: 0.25 ✓
หลอกให้สมัครแต่หาสัญญานไม่มีเลยอ่ะ.., 0.24 Ans: 0.25 ✓
ตอนนี้กลับมาใช่ได้ตามปกติ, 0.32 Ans: 1.00 X
จะย้ายเหมือนกัน โปร ais ห่วยกว่าเจ้าอื่น, 0.38 Ans: 0.00 ✓
ติดโปรหรอเปล่าค่ะ, 0.59 Ans: 0.50 ✓
โม้มากๆๆ  0877340777  ดีเเทคน่ะ โม้มากๆ, 0.14 Ans: 0.50 X
ปรับสัณญาณไม่ให้หล่มก่อนดีใหมค่ะ ขนาดบ้านอยู่ในตัวเมือง ใกล้กับเสาส่งสัณญาณยังหล่มอยู่บ่อยๆ เน็ตนี้หมุนแทบอ๊วกเลยคร้าาาา, 0.12 Ans: 0.25 ✓
Acc: 0.50
............................................................................................................................................................
Epoch

............................................................................................................................................................
Epoch -  10
Avg Accuracy (training set): 0.86
Accuracy (test set): 0.67
Show test result
ตั้งแต่ 27 Dec จนถึงตอนนี้ จะครบเดือนแล้ว Dtac บอกจะแก้ไขให้กับลูกค้า สัดท้าย ..เงียบบบบบบบ!!, 0.22 Ans: 0.00 ✓
พอไปติดต่อหาของไม่เจอแล้วให้ผมรอเป็นชั่วโมงถึงยอมคืยเอกสารให้ไม, 0.33 Ans: 0.25 ✓
ต้องจ่ายค่าเครื่อง12000กว่าบาทแล้วต้องมานั่งจ่ายรายเดือน599รวมภาษีก็หกร้อยกว่าบาทอีกตั้งเป็นปีไม่คุ้มเลยค้า, 0.28 Ans: 0.25 ✓
ซื้อมาใช้ค่ะแค้นมากโฆษณาเกินจริงอย่าไปเชื่อเน๊ตและโทรฟรี4000, 0.22 Ans: 0.25 ✓
เน็ต512 89 บาท ต้องกดยังไงค่ะ, 0.66 Ans: 0.75 ✓
เติมเงินสมัครได้ไหม?, 0.72 Ans: 0.50 X
ของผมแจ้งปัญหาการใช้การไปตั้งแต่กลางปีที่แล้ว จนตอนนี้ เครือข่ายแจ้ง no service กับ มีสัญญาณเม็ดเดียว โทรเข้าออก เล่นอินเตอร์เน็ตไม่ค่อยได้เลยครับ. โทรแจ้งคอลเซ็นเตอร์พนักงานพูดเป็นแต่ "รีเซ็ตสัญญาณ" ผู้เกี่ยวข้องตรวจสอบทีครับ, 0.12 Ans: 0.25 ✓
บ้านผมไม่มีเสาสัญญาณมาต

............................................................................................................................................................
Epoch -  13
Avg Accuracy (training set): 0.89
Accuracy (test set): 0.68
Show test result
กรุงเทพฯ ยังขี้กากเลยครับ ดีแทค
4G มีแค่ ขีดสองขีดเนี่ยนะ แก้ไขเถอะ ดีกว่าแก้ตัวนะ กำ โฆษณาจังเลย ห่วยๆๆๆๆ, 0.22 Ans: 0.25 ✓
#งานนนี้เข้าข่ายหลอกลวงประชาชนรึป่าวคะดีแทคชี้แจงด่วนนน, 0.31 Ans: 0.25 ✓
แบบนี้ต้องฉลองใหญ่ๆ
"#งดเก็บค่าบริการ1เดือน"
#โอ้ววโหววดีใจจังเดือนนี้ไม่ต้องจ่ายค่าโทรศัพท์รายเดือนเย้ๆๆๆๆๆ#$&€£₩&#$/^&%, 0.99 Ans: 1.00 ✓
อยากให้มีโปรความเร็วเหมือนกับของเน็ตบ้านเยอะๆบ้างน่าจะเหมาจ่ายไปเลยโดยที่ไม่จำกัดความเร็วก็น่าจะดีนะ, 0.44 Ans: 0.75 X
เน็นผมช้าต้องทำไงอะสมัก89บาทแต่ดูยูทูปไม่ได้เล่นgooleก็ไม่ได้แต่เล่นเฟสได้คับทำไงดี, 0.60 Ans: 0.25 X
ลดสปีกมั้ยค้ะ, 0.59 Ans: 0.75 X
ผมเห็นพนักงานaisในคลิปโดนัทที่โลตัสนครนายกยกพวกไปต่อว่าลูกค้าโดนัทบอกเลยครับว่าaisบกพร่องด้านการอบรมพนักงานมากๆผมเบอร์0-31938145ขอใช้aisแค่เดือนสุดท้ายเพราะเคยเจอ

............................................................................................................................................................
Epoch -  16
Avg Accuracy (training set): 0.90
Accuracy (test set): 0.68
Show test result
สัญญาณขึ้นRHตลอด3วันแล้วแต่ก่อนไม่เป็นปรับแต่เครือข่ายได้H+แป๊ปเดียวก็เป็นอีกเขตมาบตาพุดครับมือถือiQ xA, 0.24 Ans: 0.25 ✓
ใช้ 99 หมุนส่งงานไม่ไป้ลยค่ะ, 0.41 Ans: 0.25 ✓
ต้องจองใว้หรือเปล่าคัฟแลาวหมดเขตวันใหนพอดีสนใจไอโฟน6เอสพลัสอ่ะคัฟตอนนี้อยู่อมตะชลบุรีสาขาที่พลีสมอลไม่มีเครื่องอ่ะคัฟสามามรถจองได้ป่าวคัฟ, 0.75 Ans: 0.75 ✓
ทำไหมรุ่น4Gมีปัญหามากระบบไม่ค่อยทำงานเข้าแอ็ปก็ยากครับ, 0.27 Ans: 0.25 ✓
ใช้งานอยู่บนถนนรัชดาภิเษก 
พระราม 9 หมุนแล้ว หมุนอีก หลุดแล้ว หลุดอีก แย่เสมอต้นเสมอปลาย, 0.29 Ans: 0.25 ✓
พัฒนา 4G แล้ว  3G ก็ดูแลด้วย ไม่มีใครซื้อมือถือเพื่อตามระบบใหม่ ๆ ได้ตลอดหรอกนะ, 0.16 Ans: 0.25 ✓
ไหลมาปรับปรุงที่ อ.เมืองสมุทรปราการต่อหน่อยได้ไหมคะ บอดเลยค่ะ😓, 0.49 Ans: 0.25 X
ถ้าจะซื้อคือต้องจ่ายค่าเครื่อง 13990+vat7%+ล่วงหน้าอีก1500 แค่นี้ใช่มั

Accuracy (test set): 0.69
Show test result
รู้สึกยังแพงกว่าอยู่ดี ช่วยแนะนำวิธีย้ายค่ายหน่อยครับ, 0.46 Ans: 0.12 X
I6ค่ะ มีโปรอะไรบ้าง, 0.74 Ans: 0.75 ✓
กล่องฟรี แล้วค่าจานเท่าไรครับ ผมเปิดเบอร์ใหม่ ตอบมาครับ, 0.69 Ans: 0.50 X
เน็ตก็เน่าทุกวัน (ส่วนตัวผมเองใช้ดีเทดอยู่, 0.29 Ans: 0.00 ✓
A37 ต้องย้ายค่าย/เปิดเบอร์ใหม่ค่ะ ที่ต้องชำระแค่1,605บาท, 0.63 Ans: 0.75 X
เนตบ้าน จะหลุดไปไหนนนน ไม่หลุดก็ช้า จะทำไรก็ไม่ได้ทำเซ่งว่ะ เงินค่าเน็ตก็จ่ายทุกเดือนไม่เคยขาด เป้นตั้งเเต่บ่ายจนตอนนี้ไม่หลุดละเเต่ ช้ามากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ จ่ายไปราคาเต็มเล่นไม่คุ้มอ่ะ เซงๆ, 0.19 Ans: 0.25 ✓
ผมจะเอาซิมไปอัพเป็น4Gที่ศูนย์ดีแทคมันเสียค่าอะไรไหมครับ, 0.41 Ans: 0.75 X
วันนี้น่าให้ใช้เนตฟรีนะครับ อยากดูถ่ายทอดขบวนพระศพ, 0.82 Ans: 0.75 ✓
มีขายเหรอ อยากได้ค่ะ, 0.71 Ans: 0.75 ✓
รย้ายค่ายเหอะ นโยบายติงต๋องต้องให้บัตรตัวจิง Goodbye TRUE MoveH #GBTMH, 0.40 Ans: 0.25 ✓
Acc: 0.60
.....................................................................................................................................................

............................................................................................................................................................
Epoch -  22
Avg Accuracy (training set): 0.92
Accuracy (test set): 0.66
Show test result
ไม่เห็นลื่นเลย, 0.40 Ans: 0.25 ✓
ถ้าจะแลกซื้อ เจ2 ต้องเสียเท่าไรค้ะ, 0.63 Ans: 0.75 X
ไปเปลี่ยนเป็นเติมเงินมาแล้วคับ ซุกยอด จ่ายตามแอ้พแล้วยังมียอดค้าง โปร 299 จ่ายจริง 4xx คือไรคับ, 0.52 Ans: 0.25 X
แน่ใจว่าดีแล้วหรอ??? สมัครแบบรายเดือน4 ร้อยกว่าบาททท ตอนนี้ทำเหี้ยใรได้ นอกจาก ตอบเเชตเฟสไลน์ โพสรุป ดูอะไรที่เป็นวิดีโอไม่ได้เลยยย อย่าเสียเวลามาโฆษณาเลยจ้าาา, 0.30 Ans: 0.00 ✓
เก็ยค่าบริการ ซำ้แล้วซำ้อีก โทรมาทวง ก็แจ้งให้ทราบแล้วว่าเดือนที่แล้วจ่ายแล้วมีใบเสร็จรับเงินด้วย ยังมีหน้ามาบอกว่าจะลดให้ ไม่ฟ้อง กสทช ก็บุญหัวแล้วยังมาระงับโทรออกอีก เดียวว่างเจอกันที่ กสทช นะ dtac เก็บบิลมั่ว, 0.31 Ans: 0.25 ✓
เปิดได้ทุกที่พอปิดไม่มีที่ให้ปิดใช้มา7ปี ให้เข้าศูนใหญ่อย่างเดียวโอ้ย, 0.52 Ans: 0.25 X
ช่วยตอบใน inbox ด้วยครับ, 0.49 Ans: 0.50 ✓
ครอบคลุม ถุย, 0

............................................................................................................................................................
Epoch -  25
Avg Accuracy (training set): 0.91
Accuracy (test set): 0.68
Show test result
สัญญาณแย่มากค่ะGPS มั่วตลอด หาสัญญาณไม่เจอซะส่วนใหญ่, 0.26 Ans: 0.25 ✓
รอใจจะขาดละเมื่อไหร่จะย้ายค่ายให้ทีวะ, 0.33 Ans: 0.25 ✓
พอดีเพี่อนใช้เป็นเติมเงินถ้าเปลี่ยนเป็นใช้รายเดีอนจะมีส่วนลดอะไรบ้างคะใช้. Dtac.  มา7ปีแล้วคะ, 0.83 Ans: 0.75 ✓
จอง vivo v5 มาครึ่งเดือนแล้ว ยังไม่ได้เครื่องเลย ที่ีร้านดีแทคบอกว่า รอทางดีแทคใหญ่ส่งมา ทำไมนานจัง, 0.69 Ans: 0.75 ✓
ผมเป็นลูกค้า 4 smart choice มีโปรซื้อไอโฟนยังไงบ้างครับ, 0.69 Ans: 0.75 ✓
บ้านผมยังขึ้น E อยู่เลยคับ
ไหนบอกว่าแรงคับบ, 0.21 Ans: 0.25 ✓
เมื่อไหร่จะพัฒนาให้4g ครอบคลุมทุกพื้นที่สักที เงินที่ลูกค้าจ่ายไปกี่ล้านบาทมันไม่พอที่จะพัฒนาเลยหรอ???, 0.24 Ans: 0.00 ✓
เดี๋ยวนี้มีโปรได้โทรศัพท์ฟรีไหมครับ, 0.51 Ans: 0.50 ✓
เราใช้ 449 ใช้เน็ตตลอด ดูยูทูปทั้งวันทั้งคืนก็ยังลื่นปื๊ด ลื่นปื๊ด ไม่มีสะดุดเลยค่ะ, 0

............................................................................................................................................................
Epoch -  28
Avg Accuracy (training set): 0.92
Accuracy (test set): 0.68
Show test result
ซื้อมือถือพร้อมโปรไม่จ่ายค่าล่วงหน้าได้ไหม, 0.56 Ans: 0.75 X
เพิ่งสมัครโปรเสริมไปเมื่อวาน 5gb 99บ. วันนี้เจอโปรนี้
ครั้งก่อน กำลังเดินออกจากโรงหนังมีข้อความส่งมาบอกว่า"ท่านได้รับสิทธิ์ซื้อตั๋วหนังในราคาพิเศษเรื่องที่เพิ่งจะดูจบและกำลังเดินออกมา"

เจ็บใจ, 0.24 Ans: 0.25 ✓
Dtac รับประกันความกาก ตลอดอายุการใช้งาน, 0.29 Ans: 0.25 ✓
เลือกสายจากคนไกลๆ เพราะแม้จะอยู่ไกลกันแค่ไหน ก็เหมือนอยู่ใกล้กันแค่เอื้อมเพราะเราโทรคุยกันทุกวัน เชื่อมความสัมพันธ์แนบแน่นด้วยสัญญาณของ AIS #AISLoveMom, 0.37 Ans: 0.50 X
ช่วงนี่ดีแทคสายหลุดบ่อยมาก, 0.27 Ans: 0.25 ✓
ห้องเราอยุ่ชั้นแรกตึก4ชั้ลสัญญานขีดเดียวไม่มี..โททีนึงเล่นเน็ตทีนึง..วิ่งออกนอกตึกค่ะ..ยกเลิกบริการไปละ..แล้วคุณจะรักดีแทค..สโลแกน..ไม่ว่าค่ะ..แต่ไม่ขอใช้ละเสียรายเดือนเน็ตไปป่าววววเลย, 0.26 Ans: 0.25 ✓
น่ารัก

In [307]:
import deepcut as dc
def print_sentence_sentiment(sentences,verbose=False):
    comment = dc.tokenize(sentences)
    batch_X = np.zeros([1,max_seq_len,k+2])

    for j in range(max_seq_len):
        if(max_seq_len-j>len(comment)): # Should not be word yet
            if(verbose):print("PAD",end=' ')
            batch_X[0][j][-1] = 1
        else: 
            if(verbose):print(comment[j-(max_seq_len-len(comment))],end=' ')
            try:
                batch_X[0][j] = np.concatenate((model[comment[j-(max_seq_len-len(comment))]],np.zeros([2])))
            except Exception:
                if(verbose):print("UNK",end=' ')
                batch_X[0][j] = np.concatenate((np.zeros([k+1]),[1]))

    print(batch_X.shape)
    print(batch_X.sum())
    all_test_pred = sess.run(allpredictions,{data:batch_X})
    cur_seq_pred = all_test_pred[0,-len(comment):]
    for i,word in enumerate(comment): 
        printSegmentColor(word,cur_seq_pred[i])
    print()
    final_prediction = sess.run(prediction,{data:batch_X})
    print("Predict: %.2f"%final_prediction)

print_sentence_sentiment("ควยกูด่ามึงอยู่เนี่ยย กากมาก ไม่ยอมปรับปรุง")

(1, 75, 802)
-38.2691227894
ควยกูด่ามึงอยู่เนี่ยย กากมาก ไม่ยอมปรับปรุง
Predict: 0.69


In [242]:
print_sentence_sentiment("สัญญาณไม่ดี ไปตวจ ดับสนิท เลยเลิกใช้มานานแล้ว แต่รักอั้มนะคะ ชอบมาก ใช้ดีแทคมา7-8ปี แต่ต้องเปลี่ยนเพราะเน็ตไม่ดี ")

สัญญาณไม่ดี ไปตวจ ดับสนิท เลยเลิกใช้มานานแล้ว แต่รักอั้มนะคะ ชอบมาก ใช้ดีแทคมา7-8ปี แต่ต้องเปลี่ยนเพราะเน็ตไม่ดี 
Predict: 0.65


In [278]:
print_sentence_sentiment("รักอั้มนะคะ ชอบมาก ใช้ดีทคมา7-8ปี แต่ต้องเปลี่ยนเพราะ เน็ตไม่ดี สัญญาณไม่ดี ไปตจว ดับสนิท")

รักอั้มนะคะ ชอบมาก ใช้ดีทคมา7-8ปี แต่ต้องเปลี่ยนเพราะ เน็ตไม่ดี สัญญาณไม่ดี ไปตจว ดับสนิท
Predict: 0.11


In [298]:
print_sentence_sentiment("ใช้ดีทคมา7-8ปี แต่ต้องเปลี่ยนเพราะ เน็ตไม่ดี สัญญาณไม่ดี ไปตจว ดับสนิท แต่รักอั้มนะคะ ชอบมาก")

ใช้ดีทคมา7-8ปี แต่ต้องเปลี่ยนเพราะ เน็ตไม่ดี สัญญาณไม่ดี ไปตจว ดับสนิท แต่รักอั้มนะคะ ชอบมาก
Predict: 0.09


In [266]:
print_sentence_sentiment("เครื่องฟรีรายเดือนราคาเบามีไหมค่ะ")

เครื่องฟรีรายเดือนราคาเบามีไหมค่ะ
Predict: 0.49


In [271]:
print_sentence_sentiment("ใช้ Dtac มา 12 ปีแหละ เน็ตไม่มีหลุด")

ใช้ Dtac มา 12 ปีแหละ เน็ตไม่มีหลุด
Predict: 0.30


In [26]:
saver = tf.train.Saver()
savePath = saver.save(sess, '../model/lstm_regression.ckpt')
print(savePath)

ValueError: At least two variables have the same name: Variable_1/Adam

In [23]:
saver = tf.train.Saver()
saver.save(sess, 'lstm_sentiment_model_210717')

'lstm_sentiment_model_210717'

In [24]:
saver = tf.train.import_meta_graph('../model/lstm_regression.ckpt.meta')
saver.restore(sess,  '../model/lstm_regression.ckpt')
# access a variable from the saved Graph, and so on:
sess = sess.run(init_op)

INFO:tensorflow:Restoring parameters from ../model/lstm_regression.ckpt


In [25]:
sess.close()
#https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model

AttributeError: 'NoneType' object has no attribute 'close'